# 1. Create schema in HMS

In [28]:
%%sql
CREATE DATABASE nyc100 LOCATION 's3a://warehouse/nyc100';

++
||
++
++

# 2. Spark example with Python API

In [29]:
TAXIS_TABLE="hms.nyc100.taxis"

## 2.1 Create Hive table

In [30]:
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType

schema = StructType([
  StructField("vendor_id", LongType(), True),
  StructField("trip_id", LongType(), True),
  StructField("trip_distance", FloatType(), True),
  StructField("fare_amount", DoubleType(), True),
  StructField("store_and_fwd_flag", StringType(), True)
])

df = spark.createDataFrame([], schema)
df.writeTo(TAXIS_TABLE).create()

## 2.2 Write some data to Hive table

In [31]:
schema = spark.table(TAXIS_TABLE).schema
data = [
    (1, 1000371, 1.8, 15.32, "N"),
    (2, 1000372, 2.5, 22.15, "N"),
    (2, 1000373, 0.9, 9.01, "N"),
    (1, 1000374, 8.4, 42.13, "Y")
  ]
df = spark.createDataFrame(data, schema)
df.writeTo(TAXIS_TABLE).append()

## 2.3 Read Hive table

In [32]:
spark.read.table(TAXIS_TABLE).show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
|        1|1000371|          1.8|      15.32|                 N|
|        2|1000372|          2.5|      22.15|                 N|
|        2|1000373|          0.9|       9.01|                 N|
|        1|1000374|          8.4|      42.13|                 Y|
+---------+-------+-------------+-----------+------------------+



# 3. Spark SQL example

In [34]:
%%sql
CREATE TABLE hms.nyc100.taxis_new
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id);

++
||
++
++

In [2]:
%%sql
INSERT INTO hms.nyc100.taxis_new
VALUES (1, 1000371, 1.8, 15.32, 'N'), (2, 1000372, 2.5, 22.15, 'N'), (2, 1000373, 0.9, 9.01, 'N'), (1, 1000374, 8.4, 42.13, 'Y');

++
||
++
++

In [3]:
%%sql
SELECT * FROM hms.nyc100.taxis_new;

vendor_id,trip_id,trip_distance,fare_amount,store_and_fwd_flag
1,1000371,1.7999999523162842,15.32,N
1,1000374,8.399999618530273,42.13,Y
2,1000372,2.5,22.15,N
2,1000373,0.8999999761581421,9.01,N
1,1000371,1.7999999523162842,15.32,N
1,1000374,8.399999618530273,42.13,Y
2,1000372,2.5,22.15,N
2,1000373,0.8999999761581421,9.01,N
